In [9]:
import os
import pandas as pd
import h3
import folium
from collections import defaultdict
import tqdm
from tqdm import tqdm
import random
from itertools import combinations
import random

In [16]:
data_path = '/disk_5/grid_v2_vm_data/disk_4/pre_processed_data/state_grp_KARNATAKA/split_train_test_valid_state_KARNATAKA'
train_file = '560035_full_train.csv'
file_path = os.path.join(data_path, train_file)
train_df = pd.read_csv(file_path)

In [17]:
train_df.head()

addr_hash addr1  \
0  45dbb8e12c6648c5d8b1efdbe642afba   NaN   
1  11bbbce1781e38fcfd05e9e5224d0ab5     0   
2  11bbbce1781e38fcfd05e9e5224d0ab5     0   
3  11bbbce1781e38fcfd05e9e5224d0ab5     0   
4  11bbbce1781e38fcfd05e9e5224d0ab5     0   

                                       addr2       city      state  pincode  \
0                                        NaN  Bangalore         KA   560035   
1  14, 4th Cross Road, Phase 2, Amrita Nagar  Bengaluru  Karnataka   560035   
2  14, 4th Cross Road, Phase 2, Amrita Nagar  Bengaluru  Karnataka   560035   
3  14, 4th Cross Road, Phase 2, Amrita Nagar  Bengaluru  Karnataka   560035   
4  14, 4th Cross Road, Phase 2, Amrita Nagar  Bengaluru  Karnataka   560035   

  country        lat        lng                      time  accuracy_level  \
0   India  12.896057  77.717899  2022-02-12 18:27:22+0530        7.216000   
1   India  12.893634  77.678591  2021-05-02 11:30:36+0530      108.584999   
2   India  12.893668  77.678575  2021-06-19 16:21:30+0530       23.576000   
3   India  12.895894  77.681195  2021-06-28 10:02:21+0530       24.271999   
4   India  12.893586  77.678383  2021-08-03 15:40:24+0530       20.356001   

                                original_address  \
0                                             ,    
1  0 , 14, 4th Cross Road, Phase 2, Amrita Nagar   
2  0 , 14, 4th Cross Road, Phase 2, Amrita Nagar   
3  0 , 14, 4th Cross Road, Phase 2, Amrita Nagar   
4  0 , 14, 4th Cross Road, Phase 2, Amrita Nagar   

                              cleaned_address  lookup_lat  lookup_lng  \
0                                         NaN   12.896057   77.717899   
1   0 14 4 th cross road phase 2 amrita nagar   12.893668   77.678575   
2   0 14 4 th cross road phase 2 amrita nagar   12.893668   77.678575   
3   0 14 4 th cross road phase 2 amrita nagar   12.893668   77.678575   
4   0 14 4 th cross road phase 2 amrita nagar   12.893668   77.678575   

   confidence_score  no_of_delivereies  
0                 0                  1  
1                 1                  8  
2                 1                  8  
3                 1                  8  
4                 1                  8

In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648048 entries, 0 to 648047
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   addr_hash          648048 non-null  object 
 1   addr1              648042 non-null  object 
 2   addr2              580250 non-null  object 
 3   city               648048 non-null  object 
 4   state              642520 non-null  object 
 5   pincode            648048 non-null  int64  
 6   country            647918 non-null  object 
 7   lat                648048 non-null  float64
 8   lng                648048 non-null  float64
 9   time               648048 non-null  object 
 10  accuracy_level     648048 non-null  float64
 11  original_address   648048 non-null  object 
 12  cleaned_address    648047 non-null  object 
 13  lookup_lat         648048 non-null  float64
 14  lookup_lng         648048 non-null  float64
 15  confidence_score   648048 non-null  int64  
 16  no

In [18]:
cleaned_df = train_df.dropna()

In [19]:
cleaned_df.head()

addr_hash addr1  \
1  11bbbce1781e38fcfd05e9e5224d0ab5     0   
2  11bbbce1781e38fcfd05e9e5224d0ab5     0   
3  11bbbce1781e38fcfd05e9e5224d0ab5     0   
4  11bbbce1781e38fcfd05e9e5224d0ab5     0   
5  11bbbce1781e38fcfd05e9e5224d0ab5     0   

                                       addr2       city      state  pincode  \
1  14, 4th Cross Road, Phase 2, Amrita Nagar  Bengaluru  Karnataka   560035   
2  14, 4th Cross Road, Phase 2, Amrita Nagar  Bengaluru  Karnataka   560035   
3  14, 4th Cross Road, Phase 2, Amrita Nagar  Bengaluru  Karnataka   560035   
4  14, 4th Cross Road, Phase 2, Amrita Nagar  Bengaluru  Karnataka   560035   
5  14, 4th Cross Road, Phase 2, Amrita Nagar  Bengaluru  Karnataka   560035   

  country        lat        lng                      time  accuracy_level  \
1   India  12.893634  77.678591  2021-05-02 11:30:36+0530      108.584999   
2   India  12.893668  77.678575  2021-06-19 16:21:30+0530       23.576000   
3   India  12.895894  77.681195  2021-06-28 10:02:21+0530       24.271999   
4   India  12.893586  77.678383  2021-08-03 15:40:24+0530       20.356001   
5   India  12.893634  77.678591  2021-05-02 11:30:36+0530      108.584999   

                                original_address  \
1  0 , 14, 4th Cross Road, Phase 2, Amrita Nagar   
2  0 , 14, 4th Cross Road, Phase 2, Amrita Nagar   
3  0 , 14, 4th Cross Road, Phase 2, Amrita Nagar   
4  0 , 14, 4th Cross Road, Phase 2, Amrita Nagar   
5  0 , 14, 4th Cross Road, Phase 2, Amrita Nagar   

                              cleaned_address  lookup_lat  lookup_lng  \
1   0 14 4 th cross road phase 2 amrita nagar   12.893668   77.678575   
2   0 14 4 th cross road phase 2 amrita nagar   12.893668   77.678575   
3   0 14 4 th cross road phase 2 amrita nagar   12.893668   77.678575   
4   0 14 4 th cross road phase 2 amrita nagar   12.893668   77.678575   
5   0 14 4 th cross road phase 2 amrita nagar   12.893668   77.678575   

   confidence_score  no_of_delivereies  
1                 1                  8  
2                 1                  8  
3                 1                  8  
4                 1                  8  
5                 1                  8

In [21]:
unique_df = cleaned_df.drop_duplicates(subset=['cleaned_address'])

print(f'Original DataFrame shape: {train_df.shape}')
print(f'Cleaned DataFrame shape: {cleaned_df.shape}')
print(f'Unique DataFrame shape: {unique_df.shape}')

Original DataFrame shape: (648048, 17)
Cleaned DataFrame shape: (574774, 17)
Unique DataFrame shape: (196972, 17)


In [22]:
addresses = unique_df['cleaned_address'].tolist()
print(len(addresses))
lat_lngs = unique_df[['lat', 'lng']].values.tolist()
print(len(lat_lngs))

196972
196972


In [23]:
addresses[:10]

[' 0 14 4 th cross road phase 2 amrita nagar',
 ' 0 29 janatha colony doddakannalli',
 ' 0 81 sri sai fancy store',
 ' 0 a wipro it sez project kodathi village varthur hobli',
 ' 0 birlamant shriam chirping wood villiyammant layout haralur enclave layout bengal ura 560035 ',
 ' 0 fast n fresh super market junnasandra road and sajapur main road',
 ' 0 janatha colony carmelaram',
 ' 0 kodathi get',
 ' 0 kpr elite kaikondrahalli',
 ' 0 near aet circle lakasmi hardvear doddakannalli sarjapur main road bengaluru 560035 ']

In [24]:
lat_lngs[:10]

[[12.8936336, 77.6785909],
 [12.913098800000002, 77.698461],
 [12.883179, 77.6956343],
 [12.8987, 77.7138608],
 [12.9089839, 77.6681863],
 [12.9082538, 77.6834946],
 [12.9061864, 77.7034767],
 [12.8922406, 77.7187151],
 [12.9085819, 77.676254],
 [12.9091267, 77.6934067]]

In [25]:
data = {
    'Address': addresses,
    'Latitude': [lat for lat, lng in lat_lngs],
    'Longitude': [lng for lat, lng in lat_lngs]
}

df = pd.DataFrame(data)

df.to_csv('my_data.csv', index=False)


In [26]:
import pandas as pd

# Load the DataFrame from the CSV file
df = pd.read_csv('my_data.csv')

# Display the DataFrame
print(df.head())

                                             Address   Latitude  Longitude
0          0 14 4 th cross road phase 2 amrita nagar  12.893634  77.678591
1                  0 29 janatha colony doddakannalli  12.913099  77.698461
2                           0 81 sri sai fancy store  12.883179  77.695634
3   0 a wipro it sez project kodathi village vart...  12.898700  77.713861
4   0 birlamant shriam chirping wood villiyammant...  12.908984  77.668186


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196972 entries, 0 to 196971
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Address    196972 non-null  object 
 1   Latitude   196972 non-null  float64
 2   Longitude  196972 non-null  float64
dtypes: float64(2), object(1)
memory usage: 4.5+ MB


In [28]:
addresses = df['Address'].tolist()
lat_lng = df[['Latitude', 'Longitude']].values.tolist()

print("Addresses:", addresses[:10])
print("Lat/Lng:", lat_lng[:10])

Addresses: [' 0 14 4 th cross road phase 2 amrita nagar', ' 0 29 janatha colony doddakannalli', ' 0 81 sri sai fancy store', ' 0 a wipro it sez project kodathi village varthur hobli', ' 0 birlamant shriam chirping wood villiyammant layout haralur enclave layout bengal ura 560035 ', ' 0 fast n fresh super market junnasandra road and sajapur main road', ' 0 janatha colony carmelaram', ' 0 kodathi get', ' 0 kpr elite kaikondrahalli', ' 0 near aet circle lakasmi hardvear doddakannalli sarjapur main road bengaluru 560035 ']
Lat/Lng: [[12.8936336, 77.6785909], [12.913098800000002, 77.698461], [12.883179, 77.6956343], [12.8987, 77.7138608], [12.9089839, 77.6681863], [12.9082538, 77.6834946], [12.9061864, 77.7034767], [12.892240600000001, 77.7187151], [12.9085819, 77.676254], [12.9091267, 77.6934067]]


In [31]:
def generate_h3_cells(lat_lngs, resolution):
    return [h3.geo_to_h3(lat, lng, resolution) for lat, lng in lat_lngs]

resolution = 10
h3_cells = generate_h3_cells(lat_lngs, resolution)
len(h3_cells)

196972

In [32]:
def is_valid_h3_cell(cell):
    try:
        h3.h3_get_resolution(cell)
        return True
    except h3.H3CellError:
        return False

def generate_h3_cells(lat_lngs, resolution):
    return [h3.geo_to_h3(lat, lng, resolution) for lat, lng in lat_lngs]

def generate_triplets_on_the_fly(h3_cells, K, T, D, resolution):
    cell_to_points = defaultdict(list)
    h3_cells_set = set(h3_cells)
    
    for index, cell in enumerate(h3_cells):
        if is_valid_h3_cell(cell):
            cell_to_points[cell].append(index)

    triplets = []
    for cell, points in tqdm(cell_to_points.items(), desc="Generating Triplets"):
        if len(points) > 1:
            
            current_resolution = h3.h3_get_resolution(cell)
            if current_resolution >= resolution:
                try:
                    parent_cell = h3.h3_to_parent(cell, resolution - 1)
                except (h3.H3CellError, h3.H3ResolutionError):
                    continue

                rings = []
                for ring_level in range(1, D + 1):
                    if ring_level == 1:
                        ring = h3.k_ring(parent_cell, 2) - h3.k_ring(parent_cell, 1) # Correct 1-skip neighbor logic
                    else:
                        previous_ring = h3.k_ring(parent_cell, ring_level)
                        ring = h3.k_ring(parent_cell, ring_level + 1) - previous_ring
                    rings.append(ring)

            for point in points:
                positive_points = [p for p in points if p != point]
                if positive_points:
                    sampled_positives = random.sample(positive_points, min(K, len(positive_points)))
                    for positive_sample in sampled_positives:
                        for ring_level in range(1, D + 1):
                            valid_negative_samples = set()
                            for neighbor in rings[ring_level - 1]:
                                children = h3.h3_to_children(neighbor, resolution)
                                for child in children:
                                    if child in h3_cells_set:
                                        valid_negative_samples.add(child)
                                        if len(valid_negative_samples) >= 50:
                                            break
                                if len(valid_negative_samples) >= 50:
                                    break
                            if valid_negative_samples:
                                sampled_negatives = random.sample(valid_negative_samples, min(T, len(valid_negative_samples)))
                                for neg in sampled_negatives:
                                    neg_index = h3_cells.index(neg)
                                    triplets.append((point, positive_sample, neg_index, ring_level))

    return triplets

In [34]:
K = 2
T = 1 
D = 3
triplets = generate_triplets_on_the_fly(h3_cells, K, T, D, resolution)
print("Generated triplets:", len(triplets))

Generating Triplets: 100%|██████████| 4524/4524 [11:07<00:00,  6.77it/s] 

Generated triplets: 1165144


In [35]:
def add_hexagons_to_map(h3_cells, color, m):
    for cell in h3_cells:
        boundary = h3.h3_to_geo_boundary(cell)
        boundary = [(lat, lng) for lat, lng in boundary]
        folium.Polygon(
            locations=boundary, 
            color=color, 
            fill=True, 
            fill_color=color, 
            fill_opacity=0.0003,  
            weight=2  
        ).add_to(m)
        
def visualize_triplets_with_grid(lat_lngs, triplets, h3_cells, resolution, D):

    average_lat = sum(lat for lat, lng in lat_lngs) / len(lat_lngs)
    average_lng = sum(lng for lat, lng in lat_lngs) / len(lat_lngs)
    m = folium.Map(location=[average_lat, average_lng], zoom_start=13)

    colors = ['orange', 'red', 'purple', 'blue']
    
    for triplet in triplets:

        anchor_idx, positive_idx, negative_idx, ring_level = triplet


        anchor_coord = lat_lngs[anchor_idx]
        positive_coord = lat_lngs[positive_idx]
        negative_coord = lat_lngs[negative_idx]

        folium.Marker(location=anchor_coord, popup='Anchor', icon=folium.Icon(color='blue')).add_to(m)
        folium.Marker(location=positive_coord, popup='Positive', icon=folium.Icon(color='green')).add_to(m)
        folium.Marker(location=negative_coord, popup='Negative', icon=folium.Icon(color='red')).add_to(m)

        folium.PolyLine(locations=[anchor_coord, positive_coord], color='red' , weight=1.5).add_to(m)
        folium.PolyLine(locations=[anchor_coord, negative_coord], color='blue', weight=1.5).add_to(m)

        anchor_cell = h3_cells[anchor_idx]
        add_hexagons_to_map([anchor_cell], 'blue', m)

        positive_cell = h3_cells[positive_idx]
        add_hexagons_to_map([positive_cell], 'green', m)

        parent_cell = h3.h3_to_parent(anchor_cell, resolution - 1)
        add_hexagons_to_map([parent_cell], 'yellow', m)  

        rings = []
        for ring_level in range(1, D + 1): 
            if ring_level == 1:
                ring = h3.k_ring(parent_cell, 1) - {parent_cell}
            else:
                previous_ring = h3.k_ring(parent_cell, ring_level - 1)
                ring = h3.k_ring(parent_cell, ring_level) - previous_ring
            rings.append(ring)

        for ring_level in range(1, D + 1):
            ring = rings[ring_level - 1]
            add_hexagons_to_map(ring, colors[ring_level - 1], m)

    m.save('Latex_one_sample_2_1_3_all.html')
    return m

In [35]:

if triplets:
    visualize_triplets_with_grid(lat_lngs, triplets[90:200], h3_cells, resolution, D+1)

NameError: name 'visualize_triplets_with_grid' is not defined

In [36]:
triplets[:9]

[(0, 14647, 1683, 1),
 (0, 14647, 693, 2),
 (0, 14647, 7170, 3),
 (0, 133472, 528, 1),
 (0, 133472, 32612, 2),
 (0, 133472, 935, 3),
 (775, 126868, 1131, 1),
 (775, 126868, 4911, 2),
 (775, 126868, 12775, 3)]

In [21]:
import torch
import torch.nn as nn

class CustomTripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(CustomTripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative, ring_levels):        
        distance_positive = torch.nn.functional.pairwise_distance(anchor, positive)       
        distance_negative = torch.nn.functional.pairwise_distance(anchor, negative)
        margin_adjusted = self.margin * ring_levels
        losses = torch.relu(distance_positive - distance_negative + margin_adjusted)
        
        return losses.mean()

In [26]:

anchor_embeddings = torch.randn(10, 128)  
positive_embeddings = torch.randn(10, 128)
negative_embeddings = torch.randn(10, 128)
ring_levels = torch.tensor([1, 2, 3, 1, 2, 3, 1, 2, 3, 1])  


triplet_loss_fn = CustomTripletLoss(margin=1.0)

loss = triplet_loss_fn(anchor_embeddings, positive_embeddings, negative_embeddings, ring_levels)

print("Triplet Loss:", loss.item())

Triplet Loss: 1.6533797979354858
